In [1]:
import os
import torch
from torchvision import transforms, models,datasets
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import torch.nn as nn


class CONFIG:
    IMAGE_HEIGHT = 224
    IMAGE_WIDTH = 224
    CHANNELS = 3
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    BATCH_SIZE = 32
    DROPOUT = 0.3
    MODEL_PATH = "best_model.pth"
    INFERENCE_PATH = "inference_images"
    DATASET_PATH = "dataset"

# transformation
transform = transforms.Compose([
    transforms.Resize((CONFIG.IMAGE_HEIGHT, CONFIG.IMAGE_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def load_model(checkpoint_path, num_classes):
    model = models.efficientnet_b0(weights=None)  # Do not use pretrained weights here
    model.classifier = nn.Sequential(
        nn.Dropout(p=CONFIG.DROPOUT, inplace=True),
        nn.Linear(model.classifier[1].in_features, num_classes, bias=True)
    )
    
    checkpoint = torch.load(checkpoint_path, map_location=CONFIG.DEVICE)
    model.load_state_dict(checkpoint)
    model.to(CONFIG.DEVICE)
    model.eval()
    
    return model

# dataset for inference
class InferenceDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, image_path


def predict(model, data_loader, class_names, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, image_paths in data_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            predicted_classes = [class_names[p] for p in predicted.cpu().numpy()]
            for path, pred in zip(image_paths, predicted_classes):
                predictions.append((path, pred))
    return predictions


if __name__ == "__main__":
    dataset = datasets.ImageFolder(root=CONFIG.DATASET_PATH, transform=transform)
    class_names = dataset.classes
    
    model = load_model(CONFIG.MODEL_PATH, num_classes=len(class_names))


    image_paths = [os.path.join(CONFIG.INFERENCE_PATH, f) for f in os.listdir(CONFIG.INFERENCE_PATH) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    inference_dataset = InferenceDataset(image_paths, transform=transform)
    inference_loader = DataLoader(inference_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False)


    predictions = predict(model, inference_loader, class_names, CONFIG.DEVICE)


    df_predictions = pd.DataFrame(predictions, columns=["Image_Path", "Predicted_Class"])
    print(df_predictions)

                                          Image_Path      Predicted_Class
0           inference_images\2015-07-08-12.50.07.jpg          leaf_blight
1             inference_images\D-FL-CIRI-FO.005a.jpg          common_rust
2             inference_images\D-WO-PMIN-FO.003a.jpg  powdery_mildew_leaf
3                      inference_images\Pest2367.jpg          leaf_blight
4             inference_images\septoria-up-close.jpg          leaf_blight
5  inference_images\tobacco-leaf-macro-on-white-b...            leaf_scab
6  inference_images\Tomato-late-blight-72605cba08...          leaf_blight
7                  inference_images\Vehnäpelto_6.jpg            leaf_scab
